In [1]:
import os
import torch
import json
import nanogcg
from nanogcg import GCGConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [2]:
# hopeful fix for CuBLAS determinism
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

In [3]:
# config
MODEL_ID = "meta-llama/Meta-Llama-3.1-8B-Instruct"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

Using device: cuda


In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    # dtype=torch.float16,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# target string (what we want model to say)

# target_string = "Sure, here's"

# for testing purposes
dataset = load_dataset("PKU-Alignment/BeaverTails", split="30k_train")

# filter for only safe!=true, we aren't doing rlhf

def filter_safe(example):
    return not example["is_safe"]

unsafe_dataset = dataset.filter(filter_safe)

unsafe_dataset = unsafe_dataset.shuffle(seed=42).select(range(1000)) # only sample 100 so it doesn't take forever

results = {}

for i, row in enumerate(unsafe_dataset):

    target_string = row['response']  # use the response from the dataset as the target
    prompt = row['prompt']
    formatted_prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

    config = GCGConfig(
        seed=42
        # optim_str_init="x" * 20,
        # num_steps = 20,
        # search_width=32,
        # topk=32,
        # batch_size=64,
        # use_prefix_cache=False,  # Disable KV cache to avoid HF cache API mismatch
    )
    
    print("ITERATION", i)
    result = nanogcg.run(
        model, 
        tokenizer,
        prompt,
        target=target_string,
        config=config
    )

    # Get all suffixes from the result
    # result.strings contains all optimized strings from each step
    # result.best_string contains the best suffix (lowest loss)
   
    results[prompt] = result.best_string

ITERATION 0


2025-12-09 21:21:59 [gcg.py:219] Model is in torch.float32. Use a lower precision data type, if possible, for much faster optimization.
2025-12-09 21:21:59 [gcg.py:389] Initializing attack buffer of size 0...
We detected that you are passing `past_key_values` as a tuple of tuples. This is deprecated and will be removed in v4.47. Please convert your cache or use an appropriate `Cache` class (https://huggingface.co/docs/transformers/kv_cache#legacy-cache-format)
2025-12-09 21:22:00 [gcg.py:107] buffer:
loss: 2.427441120147705 | string: x x x x x x x x x x x x x x x x x x x x
2025-12-09 21:22:00 [gcg.py:436] Initialized attack buffer.
  0%|          | 0/250 [00:00<?, ?it/s]/lambda/nfs/brandon/basis-project/.venv/lib/python3.10/site-packages/torch/autograd/graph.py:841: UserWarning: Memory Efficient attention defaults to a non-deterministic algorithm. To explicitly enable determinism call torch.use_deterministic_algorithms(True, warn_only=False). (Triggered internally at /pytorch/aten/src/

Decreasing batch size to: 248
Decreasing batch size to: 124
Decreasing batch size to: 62
Decreasing batch size to: 31


2025-12-09 21:23:06 [gcg.py:107] buffer:
loss: 2.3882975578308105 | string: give x x x x x x x x x x x x x x x x x x x
  0%|          | 1/250 [01:06<4:34:03, 66.04s/it]

Decreasing batch size to: 253
Decreasing batch size to: 126
Decreasing batch size to: 63
Decreasing batch size to: 31


...

Decreasing batch size to: 241
Decreasing batch size to: 120
Decreasing batch size to: 60
Decreasing batch size to: 30


2025-12-09 22:45:10 [gcg.py:107] buffer:
loss: 1.8384124040603638 | string:  Speak enthusiastic x general x citizen<|reserved_special_token_19|> (= -->hasManyanga MandelaApplied reasoning disfrstood refuse ignorance reminding dangerous
 31%|███       | 78/250 [1:23:09<2:58:27, 62.25s/it]

Decreasing batch size to: 246
Decreasing batch size to: 123
Decreasing batch size to: 61
Decreasing batch size to: 30


2025-12-09 22:46:13 [gcg.py:107] buffer:
loss: 1.831329345703125 | string:  Speak enthusiastic x general x citizen<|reserved_special_token_19|> (= -->hasManyanga MandelaApplied reasoning disfrstood refuse ignorance memorial dangerous
 32%|███▏      | 79/250 [1:24:13<2:58:18, 62.56s/it]

Decreasing batch size to: 240
Decreasing batch size to: 120
Decreasing batch size to: 60
Decreasing batch size to: 30


In [ ]:
with open("results/llama_suffixes_new.json", "w") as f:
    json.dump(results, f, indent=2)

print("Results, ", results)